## 朴素贝叶斯
我们试试用朴素贝叶斯完成一个中文文本分类器，一般在数据量足够，数据丰富度够的情况下，用朴素贝叶斯完成这个任务，准确度还是很不错的。

机器学习的算法要取得好效果，离不开数据，咱们先把数据加载进来看看。

### 准备数据
准备好数据，我们挑选 科技、汽车、娱乐、军事、运动 总共5类文本数据进行处理。

In [1]:
import jieba
import pandas as pd
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

随便挑几条数据看看

In [4]:
print technology[12]

　　现在家里都拉了网线，都能无线上网，一定要帮他们先登上WiFi，另外，老人不懂得流量是什么，也不知道如何开关，控制流量，所以设置好流量上限很重要，免得不小心点开了视频或者下载，电话费就大发了。


In [8]:
print car[100]

　　截至发稿时，人人车给出的处理方案仍旧是检修车辆。王先生则认为，车辆在购买时就存在问题，但交易平台并未能检测出来。因此，王先生希望对方退款。王先生称，他将找专业机构对车辆进行鉴定，并通过法律途径维护自己的权益。J256


In [6]:
print entertainment[10]

　　网综尺度相对较大原本是制作优势，《奇葩说》也曾经因为讨论的话题较为前卫一度引发争议。但《奇葩说》对于价值观的把握和引导让其中内含的“少儿不宜”只能算是小花絮。而纯粹是为了制造话题而“污”得“无节操无下限”的网综不仅让人生厌，也给节目自身招致了下架的厄运。对资本方而言，即便只从商业运营考量，点击量也分有价值和无价值，节目内容的变现能力如果建立在博眼球和低趣味迎合上，商业运营也难长久。对节目制作方与平台来说，为博一时的高点击而不顾底线不仅是砸自己招牌，以噱头吸引而来的观众与流量也是难以维持。


In [9]:
print military[10]

　　央视记者 胡善敏：我现在所处的位置是在辽宁舰的飞行甲板，执行跨海区训练和试验任务的辽宁舰官兵，正在展开多个科目的训练，穿着不同颜色服装的官兵在紧张的对舰载机进行转运。


In [10]:
print sports[10]

　　据统计，2016年仅在中国田径协会注册的马拉松赛事便达到了328场，继续呈现出爆发式增长的态势，2015年，这个数字还仅仅停留在134场。如果算上未在中国田协注册的纯“民间”赛事，国内全年的路跑赛事还要更多。


### 分词与中文文本处理

#### 停用词

In [2]:
stopwords=pd.read_csv("data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

#### 去停用词

In [3]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception,e:
            print line
            continue 

#生成训练数据
sentences = []

preprocess_text(technology, sentences, 'technology')
preprocess_text(car, sentences, 'car')
preprocess_text(entertainment, sentences, 'entertainment')
preprocess_text(military, sentences, 'military')
preprocess_text(sports, sentences, 'sports')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.256 seconds.
Prefix dict has been built succesfully.


#### 生成训练集
我们打乱一下顺序，生成更可靠的训练集

In [4]:
import random
random.shuffle(sentences)

In [5]:
for sentence in sentences[:10]:
    print sentence[0], sentence[1]

金山毒霸 专家建议 technology
解释 战友 很快 明白 航母 机电 空调 系统 重要性 阮万林 竖起 大拇指 military
实施 共享 停车 问题重重 car
中国区 独立 郑杰 权力 职能 菲克 拓展 中国 市场 合资 公司 全球化 益处 中国 市场 创新 影响 全球 市场 郑杰 中国 消费者 需求 中国 需求 全球 需求 中国 市场 全球 第一 市场 汽车 公司 中国 本土化 中国 改进 产品 这是 意识 中国 全球 地位 car
关注 爆炸 原因 电池 缺陷 technology
首发 命中 录像 回放 一旁 领队 黄庆利 激动 好家伙 发现 摧毁 露头 military
VV7c VV7s 配备 2.0 双流 涡轮 增压 缸内 直喷 发动机 车速 205km 额定功率 172kw 扭矩 360 2200 4000 rps 转速 扭矩 带来 加速性 兼顾 燃油 经济性 车辆 采用 湿式 离合 变速器 相比 传统 7AT 提升 燃油 经济性 减少 油耗 car
TechCrunch 人工智能 人才 角度 肯定 陆奇 加盟 文章 指出 百度 人工智能 人才 招募 发力 任命 微软 高管 知名 AI 专家 陆奇 集团 总裁兼 COO 百度 搜索 闻名 中国 市场 占据 主导地位 近两年来 百度 集中精力 发展 包括 无人 在内 人工智能 陆奇 加盟 成功 technology
河尾滩 边防连 巡逻 分队 天文 边防连 巡逻 分队 克服 困难 上级 指定 时间 指定 地点 会合 平时 距离 不远 难得一见 战友 神圣 边境线 相聚 那种 激动 那种 喜庆 堪比 过年 会哨 两支 巡逻 分队 返回 哨所 夕阳西下 一轮 硕大 明月 跃出 连绵 雪峰 military
宝马 i5 车型 适合 家庭 新车型 保留 i3 车型 自杀式 车门 前卫 造型 更大 实用 车身 强大 续航力 特斯拉 Model 车型 目标 报道 显示 宝马 放弃 推出 i5 车型 报道 属实 宝马 系列 车型 在短期内 只会 推出 i8 Spyder 车型 宝马 公司 2021 推出 一款 名为 iNext 自动 驾驶 汽车 car


为了一会儿检测一下咱们的分类器效果怎么样，我们需要一份测试集。

所以把原数据集分成训练集的测试集，咱们用sklearn自带的分割函数。

In [6]:
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

In [7]:
len(x_train)

65696

下一步要做的就是在降噪数据上抽取出来有用的特征啦，我们对文本抽取词袋模型特征

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    max_features=4000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

把分类器import进来并且训练

In [12]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

看看我们的准确率如何

In [13]:
classifier.score(vec.transform(x_test), y_test)

0.83474131238869353

In [14]:
len(x_test)

21899

我们可以看到在2w多个样本上，我们能在5个类别上达到83的准确率。

有没有办法把准确率提高一些呢？

我们可以把特征做得更棒一点，比如说，我们试试加入抽取2-gram和3-gram的统计特征，比如可以把词库的量放大一点。

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,4),  # use ngrams of size 1 and 2
    max_features=20000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

### 分类训练

In [16]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.87401251198684871

### 交叉验证

更可靠的验证效果的方式是交叉验证，但是交叉验证最好保证每一份里面的样本类别也是相对均衡的，我们这里使用StratifiedKFold

In [19]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

def stratifiedkfold_cv(x, y, clf_class, shuffle=True, n_folds=5, **kwargs):
    stratifiedk_fold = StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    y_pred = y[:]
    for train_index, test_index in stratifiedk_fold:
        X_train, X_test = x[train_index], x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred 

NB = MultinomialNB
print precision_score(y, stratifiedkfold_cv(vec.transform(x),np.array(y),NB), average='macro')

0.88154693235


我们做完K折的交叉验证，可以看到在5个类别上的结果平均准确度约为88%

### 我们自己来完成一个文本分类器class

In [20]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [89]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.865427645098


### SVM文本分类

我们来试试支持向量机的作用

In [23]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.84528973925750039

可以试试rbf核

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

### 换特征/模型试试

In [6]:
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


class TextClassifier():

    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [7]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.874788803142


## Facebook FastText

fasttext是facebook开源的一个词向量与文本分类工具，在学术上没有太多创新点，好处是模型简单，训练速度非常快。简单尝试可以发现，用起来还是非常顺手的，做出来的结果也不错，可以达到上线使用的标准。

简单说来，fastText做的事情，就是把文档中所有词通过lookup table变成向量，取平均后直接用线性分类器得到分类结果。fastText和ACL-15上的deep averaging network(DAN，如下图)比较相似，是一个简化的版本，去掉了中间的隐层。论文指出了对一些简单的分类任务，没有必要使用太复杂的网络结构就可以取得差不多的结果。

![](./image/fast_text.png)

fastText论文中提到了两个tricks

- hierarchical softmax
    - 类别数较多时，通过构建一个霍夫曼编码树来加速softmax layer的计算，和之前word2vec中的trick相同
- N-gram features
    - 只用unigram的话会丢掉word order信息，所以通过加入N-gram features进行补充用hashing来减少N-gram的存储

fastText做文本分类要求文本是如下的存储形式：
```
__label__2 , birchas chaim , yeshiva birchas chaim is a orthodox jewish mesivta high school in lakewood township new jersey . it was founded by rabbi shmuel zalmen stein in 2001 after his father rabbi chaim stein asked him to open a branch of telshe yeshiva in lakewood . as of the 2009-10 school year the school had an enrollment of 76 students and 6 . 6 classroom teachers ( on a fte basis ) for a student–teacher ratio of 11 . 5 1 . 
__label__6 , motor torpedo boat pt-41 , motor torpedo boat pt-41 was a pt-20-class motor torpedo boat of the united states navy built by the electric launch company of bayonne new jersey . the boat was laid down as motor boat submarine chaser ptc-21 but was reclassified as pt-41 prior to its launch on 8 july 1941 and was completed on 23 july 1941 . 
__label__11 , passiflora picturata , passiflora picturata is a species of passion flower in the passifloraceae family . 
__label__13 , naya din nai raat , naya din nai raat is a 1974 bollywood drama film directed by a . bhimsingh . the film is famous as sanjeev kumar reprised the nine-role epic performance by sivaji ganesan in navarathri ( 1964 ) which was also previously reprised by akkineni nageswara rao in navarathri ( telugu 1966 ) . this film had enhanced his status and reputation as an actor in hindi cinema . 
```
其中前面的`__label__`是前缀，也可以自己定义，`__label__`后接的为类别。

我们定义我们的5个类别分别为：
```
1:technology
2:car
3:entertainment
4:military
5:sports
```

### 生成文本格式

In [4]:
import jieba
import pandas as pd
import random

cate_dic = {'technology':1, 'car':2, 'entertainment':3, 'military':4, 'sports':5}

df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [5]:
stopwords=pd.read_csv("data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append("__label__"+str(category)+" , "+" ".join(segs))
        except Exception,e:
            print line
            continue

SyntaxError: invalid syntax (<ipython-input-5-3b0d3eefac5a>, line 11)

In [6]:
#生成训练数据
sentences = []

preprocess_text(technology, sentences, cate_dic['technology'])
preprocess_text(car, sentences, cate_dic['car'])
preprocess_text(entertainment, sentences, cate_dic['entertainment'])
preprocess_text(military, sentences, cate_dic['military'])
preprocess_text(sports, sentences, cate_dic['sports'])

random.shuffle(sentences)

NameError: name 'preprocess_text' is not defined

In [7]:
print "writing data to fasttext format..."
out = open('./data/train_data.txt', 'w')
for sentence in sentences:
    out.write(sentence.encode('utf8')+"\n")
print "done!"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("writing data to fasttext format...")? (<ipython-input-7-d16ffd2d1f81>, line 1)

### 调用fastText训练生成模型

In [9]:
import fasttext
classifier = fasttext.supervised('./data/train_data.txt', 'classifier.model', label_prefix='__label__')

ModuleNotFoundError: No module named 'fasttext'

### 对模型效果进行评估

In [10]:
result = classifier.test('./data/train_data.txt')
print 'P@1:', result.precision
print 'R@1:', result.recall
print 'Number of examples:', result.nexamples

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('P@1:', result.precision)? (<ipython-input-10-d274f7b782d6>, line 2)

### 实际预测

In [11]:
label_to_cate = {1:'technology', 2:'car', 3:'entertainment', 4:'military', 5:'sports'}

#texts = ['中新网 日电 2018 预赛 亚洲区 强赛 中国队 韩国队 较量 比赛 上半场 分钟 主场 作战 中国队 率先 打破 场上 僵局 利用 角球 机会 大宝 前点 攻门 得手 中国队 领先']
texts = ['这 是 中国 第 一 次 军舰 演习']
labels = classifier.predict(texts)
print labels
print label_to_cate[int(labels[0][0])]

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(labels)? (<ipython-input-11-4902e146aa25>, line 6)

In [12]:
labels = classifier.predict_proba(texts)
print labels

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(labels)? (<ipython-input-12-9b51658fc7a1>, line 2)

### Top K 个预测结果

In [13]:
labels = classifier.predict(texts, k=3)
print labels

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(labels)? (<ipython-input-13-dcf8a4d590d1>, line 2)

In [14]:
labels = classifier.predict_proba(texts, k=3)
print labels

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(labels)? (<ipython-input-14-a398106ad1e2>, line 2)

## fastText文本无监督学习

In [15]:
def preprocess_text_unsupervised(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append(" ".join(segs))
        except Exception,e:
            print line
            continue
#生成无监督训练数据
sentences = []

preprocess_text(technology, sentences, cate_dic['technology'])
preprocess_text(car, sentences, cate_dic['car'])
preprocess_text(entertainment, sentences, cate_dic['entertainment'])
preprocess_text(military, sentences, cate_dic['military'])
preprocess_text(sports, sentences, cate_dic['sports'])

print "writing data to fasttext unsupervised learning format..."
out = open('./data/unsupervised_train_data.txt', 'w')
for sentence in sentences:
    out.write(sentence.encode('utf8')+"\n")
print "done!"            

SyntaxError: invalid syntax (<ipython-input-15-c84fb67ce12a>, line 8)

In [16]:
import fasttext

# Skipgram model
model = fasttext.skipgram('./data/unsupervised_train_data.txt', 'model')
print model.words # list of words in dictionary

# CBOW model
model = fasttext.cbow('./data/unsupervised_train_data.txt', 'model')
print model.words # list of words in dictionary
print model['赛季']

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(model.words # list of words in dictionary)? (<ipython-input-16-e4fe503fdb6e>, line 5)

### 对比gensim的word2vec

In [17]:
def preprocess_text_unsupervised(content_lines, sentences):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append(" ".join(segs))
        except Exception,e:
            print line
            continue
#生成无监督训练数据
sentences = []

preprocess_text(technology, sentences)
preprocess_text(car, sentences)
preprocess_text(entertainment, sentences)
preprocess_text(military, sentences)
preprocess_text(sports, sentences)

model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
model.save("gensim_word2vec.model")
model.wv['赛季']

SyntaxError: invalid syntax (<ipython-input-17-6f6be01e4658>, line 8)

In [18]:
model.wv.most_similar('赛季')

NameError: name 'model' is not defined